# FewShotPainAdaptation on Google Colab (T4)
This notebook installs dependencies, verifies GPU, configures reproducibility, and runs LOSO few-shot training.

Before running, set Colab runtime to: **GPU (T4)**.

In [3]:
!nvidia-smi
!pip -q install -U pip
!pip -q install tensorflow==2.18.1 cloudpickle matplotlib numpy pandas scikit-learn scipy seaborn pydantic

Thu Feb 26 09:53:39 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
REPO_URL = "https://github.com/hhihn/FewShotPainAdaptation.git"
PROJECT_DIR = "/content/FewShotPainAdaptation"
import os
if not os.path.isdir(PROJECT_DIR):
    !git clone $REPO_URL $PROJECT_DIR
    %cd $PROJECT_DIR
else:
    %cd $PROJECT_DIR
    !git pull


/content/FewShotPainAdaptation
Already up to date.


In [5]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print('Visible GPUs:', gpus)
if not gpus:
    raise RuntimeError('No GPU detected. In Colab, set Runtime -> Change runtime type -> GPU.') # Optional, helps speed on T4.
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print('Mixed precision policy:', mixed_precision.global_policy())

Visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Mixed precision policy: <DTypePolicy "mixed_float16">


In [ ]:
# Point this to a folder containing: X_pre.npy, y_heater.npy, subjects.npy\n# Example if your data is in repo: /content/FewShotPainAdaptation/data\n# Example if in Drive: /content/drive/MyDrive/FewShotPainAdaptation/data\n
DATA_DIR = "/content/FewShotPainAdaptation/data"
import os
required = ['X_pre.npy', 'y_heater.npy', 'subjects.npy']
missing = [f for f in required if not os.path.exists(os.path.join(DATA_DIR, f))]
if missing:
    raise FileNotFoundError(f'Missing files in DATA_DIR={DATA_DIR}: {missing}')
print('Using data directory:', DATA_DIR)

In [ ]:
import json
import logging
import numpy as np
from data_loaders.pain_ds_config import PainDatasetConfig
from learner.few_shot_pain_learner import FewShotPainLearner
from utils.logger import setup_logger
logger = setup_logger('FewShotPainLearner', level=logging.INFO)
# Reproducible config
config = PainDatasetConfig(seed=42, deterministic_ops=True, k_shot=3, q_query=3,)
# Keep these small first; increase after sanity check.
NUM_EPOCHS = 3
EPISODES_PER_EPOCH = 10
VAL_EPISODES = 5
FUSION_METHODS = ['attention']
all_results = {}
for fusion_method in FUSION_METHODS:
    logger.info(f'Training with fusion method: {fusion_method}')
    learner = FewShotPainLearner(config=config, data_dir=DATA_DIR, learning_rate=1e-3, fusion_method=fusion_method, seed=config.seed,        deterministic_ops=config.deterministic_ops)
    cv_results = learner.train(num_epochs=NUM_EPOCHS, episodes_per_epoch=EPISODES_PER_EPOCH, val_episodes=VAL_EPISODES,)
    all_results[fusion_method] = cv_results
    summary = {
        fm: {
            'avg_test_acc': float(np.mean(res['test_accuracies'])),
            'std_test_acc': float(np.std(res['test_accuracies'])),
            'avg_test_loss': float(np.mean(res['test_losses'])),
        }    for fm, res in all_results.items()}
    print(json.dumps(summary, indent=2))
    with open('colab_run_results.json', 'w') as f:
        json.dump({'summary': summary, 'full': all_results}, f, indent=2)
    print('Saved results to colab_run_results.json')